# **Tarea 4 - CC6205 Natural Language Processing 📚**

**Integrantes:** Arturo Lazcano

**Fecha límite de entrega 📆:** Martes 13 de junio.

**Tiempo estimado de dedicación:** 12 horas

Bienvenid@s a la cuarta tarea del curso de Natural Language Processing (NLP).
En esta tarea estaremos tratando el problema de **tagging** (generación de secuencias de etiquetas del mismo largo que la secuencia de input), el uso de **Convolutional Neural Networks** y **Recurrent Neural Networks**, e implementaremos una red usando PyTorch.

Usen $\LaTeX$ para las fórmulas matemáticas. En la parte de programación pueden usar lo que quieran, pero la [Auxiliar 3](https://youtu.be/36WTXvg3zh0) les puede ser de *gran ayuda*.

**Instrucciones:**
- La tarea se realiza en grupos de **máximo** 2 personas. Puede ser invidivual pero no es recomendable.
- La entrega es a través de u-cursos a más tardar el día estipulado arriba. No se aceptan atrasos.
- El formato de entrega es este mismo Jupyter Notebook.
- Al momento de la revisión tu código será ejecutado. Por favor verifica que tu entrega no tenga errores de compilación.
- En el horario de auxiliar pueden realizar consultas acerca de la tarea a través del canal de Discord del curso.

Si aún no han visto las clases, se recomienda visitar los links de las referencias.

**Referencias:**

- [Tagging, and Hidden Markov Models ](http://www.cs.columbia.edu/~mcollins/cs4705-spring2019/slides/tagging.pdf) (slides by Michael Collins), [notes](http://www.cs.columbia.edu/~mcollins/hmms-spring2013.pdf), [video 1](https://youtu.be/-ngfOZz8yK0), [video 2](https://youtu.be/Tjgb-yQOg54), [video 3](https://youtu.be/aaa5Qoi8Vco), [video 4](https://youtu.be/4pKWIDkF_6Y)
- [MEMMs and CRFs](https://github.com/dccuchile/CC6205/blob/master/slides/NLP-CRF.pdf): [notes 1](http://www.cs.columbia.edu/~mcollins/crf.pdf), [notes 2](http://www.cs.columbia.edu/~mcollins/fb.pdf), [video 1](https://youtu.be/qlI-4lSUDkg), [video 2](https://youtu.be/PLoLKQwkONw), [video 3](https://youtu.be/ZpUwDy6o28Y)
- [Convolutional Neural Networks](https://github.com/dccuchile/CC6205/blob/master/slides/NLP-CNN.pdf): [video](https://youtu.be/lLZW5Fn40r8)
- [Recurrent Neural Networks](https://github.com/dccuchile/CC6205/blob/master/slides/NLP-RNN.pdf): [video 1](https://youtu.be/BmhjUkzz3nk), [video 2](https://youtu.be/z43YFR1iIvk), [video 3](https://youtu.be/7L5JxQdwNJk)

# Hidden Markov Models (HMM), Maximum Entropy Markov Models (MEMM) and Conditional Random Field(CRF) (1,5 puntos)

### Pregunta 1 (1 pt)
Para un problema de POS tagging se define el conjunto de etiquetas $S = \{ \text{DET}, \text{NOUN}, \text{VERB}, \text{ADP} \}$ y se tiene un Hidden Markov Model con los siguientes parámetros estimados a partir de un corpus de entrenamiento:

\begin{equation}
\begin{split}
q(\text{NOUN}| \text{ VERB}, \text{DET}) &= 0.3 \\
q(\text{NOUN}|\ w, \text{DET}) &= 0 \qquad \forall w \in S, w \neq \text{VERB} \\
q(\text{DET}| \text{ VERB}, \text{NOUN}) &= 0.4 \\
q(\text{DET}|\ w, \text{NOUN}) &= 0 \qquad \forall w \in S, w \neq \text{VERB} \\
e(the|\text{ DET}) &= 0.5 \\
e(pasta|\text{ NOUN}) &= 0.6
\end{split}
\end{equation}

Luego para la oración: `the man is pouring sauce on the pasta`, se tiene una tabla de programación dinámica con los siguientes valores:
\begin{equation}
\begin{split}
\pi(7,\text{DET},\text{DET})&=0.1\\
\pi(7,\text{NOUN},\text{DET})&=0.2\\
\pi(7,\text{VERB},\text{DET})&=0.01\\
\pi(7,\text{ADP},\text{DET})&=0.5
\end{split}
\end{equation}

Con esta información, calcule el valor de $\pi(8,\text{DET},\text{NOUN})$. Puede dejar el resultado expresado como una fracción.


**Respuesta**

Recordando que, gracias al algoritmo de Viterbi, la programación dinámica hace posible calcular estos valores teniendo información solamente de algunos casos anteriores.
Así,

$\pi(8,\text{DET},\text{NOUN}) = $ Máxima probabilidad de que se termine con DET, NOUN (D, N respectivamente) en la posición 8. Luego,

$\pi(8,\text{DET},\text{NOUN}) = \max_{w \in S_6} [\pi(7,w,\text{D}) \times q(\text{N}|w,\text{D}) \times e(\text{pasta}|\text{N})]$

Con esto, debemos probar los valores dados por la tabla dinámica para poder obtener el máximo, esto iteranco con $w \in S$. Notando que $q(\text{N}|w,\text{D}) = 0$ para cada $w \neq \text{VERB}$, es fácil ver que se anula cada opción, por lo que la única que no es 0 es usando $w = \text{VERB}$ resultando:

$\pi(8,\text{DET},\text{NOUN}) = \pi(7,\text{V},\text{D}) \times q(\text{N}|\text{V},\text{D}) \times e(\text{pasta}|\text{N}) = 0.01 \cdot 0.3 \cdot 0.6 = 0.0018$




### Pregunta 2 (0.5 pts)
Comente  sobre las similitudes o diferencias entre los HMMs, MEMMs y CRFs. Para esto, responda las siguientes preguntas.

#### 2.1. ¿Para qué tipo de tarea sirven? Dé dos ejemplo de este tipo de tarea y descríbalos brevemente. (0.1 pts)

**Respuesta:**

Los HMMs, MEMMs y CRFs son útiles para tareas de etiquetado secuencial. Para ello hay 2 tareas usuales que son POS tagging (Part-Of-Speech) y NER (Named Entity Recognition). Esta primera tarea busca asignar una etiqueta gramatical a cada palabra en una oración, como sustantivo, verbo, adjetivo, etc. Por otro lado, esta segunda tarea, su objetivo es identificar y clasificar menciones de entidades nombradas en un texto, como nombres de personas, organizaciones, ubicaciones, etc.

#### 2.2. ¿Qué modelos usan features? ¿Qué ventajas conlleva esto? (0.1 pts)

**Respuesta:**

Los modelos MEMMs y CRFs usan features. Posibles ventajas de esto es el mejor manejo de dependencias contextuales, poseen mejor y más rica representación y el hecho de poder elegir un vector de características entrega una mayor flexibilidad a la hora de querer ajustar lo mejor posible un modelo.

#### 2.3. ¿Cómo maneja cada uno de los modelos las palabras con baja frecuencia en el set de train? (0.1 pts)

**Respuesta:**



*   HMM's: Separa palabras dado un umbral de frecuencia. Luego, para estas palabras de baja aparición en el texto, las mapea a un conjunto finito (pequeño) dependiendo del tipo de palabra. Por ejemplo, "90" sería mapeado a una clase tipo "TwoDigitsNumber"
*   MEMM's y CRF's: Extracción de características adicionales (por ejemplo, morfológica, léxica, sintáctica, entre otras), aprovechamiento de la información contextual e incluso técnicas de suavizado.



#### 2.4. ¿Qué le permite a los CRF realizar decisiones globales? ¿Qué diferencia con respecto a los MEMMs permite lograr esto? ¿Por qué los HMMs tampoco son capaces de tomar decisiones globales? (0.1 pts)

**Respuesta:**

Esto se debe a que la normalización es global, pues involucra a la suma de todas las posibles secuencias de tag $S^m$, mientras que a los modelos MEMMs, debido a que realizan una normalización local, se pierde información pudiendo incluso clasificar incorrectamente el tipo de una palabra.
Por otro lado, los modelos HMM son locales, esto debido a su definición, ya que aprovechan técnicas de cadenas de Markov, las cuales poseen pérdida de memoria, haciendo estos modelos locales en cierta vecindad de alguna palabra objetivo.

#### 2.5 Dado una secuencia de $x_1, ..., x_m$ ¿Cuántas posibles secuencias de etiquetas se pueden generar para un conjunto de etiquetas $S$ con $|S|=k$ ? ¿Analizarlas todas sería computacionalmente tratable? (0.1 pts)

**Respuesta:**

Si el conjunto de etiquetas posee cardinal $k$, cada palabra tiene $k$ posibles etiquetas, por lo que en una secuencia de $m$ palabras, el conjunto de todas las posibles secuancias tiene cardinal $k^m$.
Viendo esto en un ejemplo sencillo, para una secuencia de 10 palabras y un conjunto de etiquetas con cardinal 5, el conjunto de todas las posibles etiquetas tiene cardinal $5^{10} = 9.765.625$, lo cual lo convierte en algo computacionalmente difícil de tratar, ya que esto tiene un crecimiento exponencial en el largo de la secuencia. Esto afecta directamente pues en los problemas prácticos, se tienen corpus de un largo mucho mayor a 10 como en el ejemplo anterior.

# Convolutional Neural Networks (0,5 puntos)

### Pregunta 3 (0,5 puntos)

Considere la frase $w_{1..7}=$ `El agua moja y el fuego quema` $=[El, agua, moja, y, el, fuego, quema]$.

La siguiente matriz de embeddings, donde la i-ésima fila corresponde al vector de embedding de la i-ésima palabra, ordenadas según aparecen en la frase. (vectores de largo 2).
\begin{equation}
E = \begin{pmatrix}
2 & 2\\
0 & -2\\
0 & 1\\
-2 & 1\\
1 & 0\\
-1 & 1\\
1 & 1
\end{pmatrix}
\end{equation}

Los siguientes 3 filtros
\begin{equation}
U = \begin{pmatrix}
-1 & 1 & 0\\
1 & 1 & 0\\
0 & 0 & -1\\
1 & -1 & -1\\
-1 & -1 & 1\\
1 & 0 & -1
\end{pmatrix}
\end{equation}

Y la función de activación
\begin{equation}
tanh = \frac{e^{2x} - 1}{e^{2x} + 1}
\end{equation}

Usando estos paramátros escriba los pasos para calcular la representación (vector) resultante de aplicar la operación de convolución (sin padding) + max pooling. ¿De qué tamaño sería la ventana que debemos usar?

**Respuesta**

Dados estos parámetros, se usa a operación $\oplus w_{i:i+k-1}$ como la concatenación de los vectores $w_i, ..., w_i+k-1$ donde $i$ representa siempre la fila y pertenece al conjunto $ \{1,...,7\}$ en el ejempo de arriba.
En este caso, como la matriz $E$ tiene ancho 2, entonces $k=2$. Luego, se genera el vector de concatenaciones $x_i = \oplus w_{i:i+k-1}$. Así, con los filtros $u$ escritos en la matriz $U$, se procede a definir $p_i = g(x_i \cdot U + b)$ donde $b$ es un vector de bias. A este producto punto se le agrega el uso de una función de activación g a cada $p_i$, en este caso la tangente hiperbólica.

Hasta este punto, obtenemos $n-k+1 = 7-2+1 = 6$ vectores $p_{1:6}$. Luego de este proceso, se pasa a la etapa de pooling, la cual consiste en combinar los vectores $p_{1:m}$ en un solo vector $c$, donde al aplicar max pooling, lo que hacemos es definir $c_{[j]} = \max_{1<i\leq m} p_{i[j]}$ donde este vector final es el que captura la información importante de la secuencia original.

# Recurrent Neural Networks (1 punto)


### Pregunta 4 (0,5 puntos)
Usando los embeddings de dos dimensiones de la pregunta anteror, la oración `el fuego quema` la podemos representar por una secuencia de vectores $(\vec{x}_1,\vec{x}_2,\vec{x}_3)$, con $\vec{x}_i \in \mathbb{R}^{d_x}$ y $d_x=2$.

Tenemos una red recurrente *Elman* definidad como:
\begin{equation}
\begin{split}
\vec{s}_i &= R_{SRNN}\left (\vec{x}_i, \vec{s}_{i-1}\right ) = g \left (\vec{s}_{i-1}W^s + \vec{x}_i W^x + \vec{b}\right ) \\
\vec{y}_i &= O_{SRNN}\left(\vec{s}_i\right) = \vec{s}_i \\
\end{split}
\end{equation}
donde
\begin{equation}
\vec{s}_i, \vec{y}_i \in \mathbb{R}^{d_s}, \quad W^x \in \mathbb{R}^{d_x \times d_s}, \quad W^s \in \mathbb{R}^{d_s \times d_s}, \quad \vec{b} \in \mathbb{R}^{d_s},
\end{equation}
y los vectores de estado $s_i$ son de tres dimensiones, $ds= 3$.

Sea
\begin{equation}
\begin{split}
\vec{s}_0 &= [0,0,0]\\
W^x &= \begin{pmatrix}
0 &  0 & 1\\
1 & -1 & 0
\end{pmatrix} \\
W^s &= \begin{pmatrix}
1 & 0 &  1\\
0 & 1 & -1\\
1 & 1 &  1
\end{pmatrix} \\
\vec{b} &= [0, 0, 0] \\
g(x) &= ReLu(x) = max(0, x)
\end{split}
\end{equation}

<br>

Calcule manualmente los valores de los vectores $\vec{s}_1, \vec{s}_2,\vec{s}_3$ y de $\vec{y}_1, \vec{y}_2,\vec{y}_3$.

**Respuesta**

$  \vec{s}_1 = \max(0, \vec{s}_0W^s + \vec{x}_1W^x + \vec{b})   $

$  \vec{s}_2 = \max(0, \vec{s}_1W^s + \vec{x}_2W^x + \vec{b})   $

$  \vec{s}_3 = \max(0, \vec{s}_2W^s + \vec{x}_3W^x + \vec{b})   $

Donde, dada la matriz de embeddings de la pregunta anterior, $\vec{x}_1 = (1,0)$, $\vec{x}_2 = (-1,1)$ y $\vec{x}_3 = (1,1)$. Así, podemos proceder a calcular directamente los vectores $\vec{s}_i$:

$\vec{s}_1 = \max(0, \vec{s}_0W^s + \vec{x}_1W^x + \vec{b}) = \max\bigg(0,
\begin{pmatrix}
1 &  0
\end{pmatrix}
\begin{pmatrix}
0 &  0 & 1\\
1 & -1 & 0
\end{pmatrix}\bigg)
= \max(0, \begin{pmatrix}
0 &  0 & 1
\end{pmatrix}) = \begin{pmatrix}
0 & 0 & 1
\end{pmatrix}$

$\vec{s}_2 = \max(0, \vec{s}_0W^s + \vec{x}_1W^x + \vec{b}) = \max\bigg(0,
\begin{pmatrix}
0 & 0 & 1
\end{pmatrix}+
\begin{pmatrix}
-1 & 1
\end{pmatrix}
\begin{pmatrix}
0 &  0 & 1\\
1 & -1 & 0
\end{pmatrix}\bigg)
= \max(0, \begin{pmatrix}
1 & -1 & 0
\end{pmatrix}) = \begin{pmatrix}
1 & 0 & 0
\end{pmatrix}$

$\vec{s}_3 = \max(0, \vec{s}_0W^s + \vec{x}_1W^x + \vec{b}) = \max\bigg(0,
\begin{pmatrix}
1 & 0 & 0
\end{pmatrix}+
\begin{pmatrix}
1 & 1
\end{pmatrix}
\begin{pmatrix}
0 &  0 & 1\\
1 & -1 & 0
\end{pmatrix}\bigg)
= \max(0, \begin{pmatrix}
2 & -1 & 1
\end{pmatrix}) = \begin{pmatrix}
2 & 0 & 1
\end{pmatrix}$

Por último, por definición, $\vec{y}_i = \vec{s}_i$, concluyendo lo pedido.





### Pregunta 5 (0.5 puntos)
¿De qué forma las RNN y las CNN logran aprender representaciones específicas
para la tarea objetivo? Compare la forma en que las RNN y las CNN aprenden con los modelos que usan *features* diseñadas manualmente.

**Respuesta**

*   RNN: Estas son capaces de capturar y modelar dependencias a largo plazo en secuencias. Esto se logra mediante la incorporación de conexiones recurrentes, que permiten que la información fluya hacia adelante y hacia atrás en la red a través del tiempo. Para esto, la red hace múltiples cálculos de valores que dependen de su capa anterior junto con embedding de la i-ésima palabra tal como se puede ver en la pregunta anterior (en el caso RNN Elman).

*   CNN: Estas utilizan filtros convolucionales para escanear localmente las características en los datos de entrada. Estos filtros convolucionales se aplican de manera repetida a diferentes regiones del espacio de entrada, generando mapas de características que resaltan patrones y características específicas, además existen técnicas como pooling que resultan en la obtención de las features más importantes en un texto.
Son especialmente útiles para tareas de visión por computadora que involucran datos estructurados en forma de imágenes o mapas de características.

Así, ambos modelos aprenden features de manera automática, en contraparte con, por ejemplo, los modelos MEMM's y CRF's que tienen features diseñadas manualmente ($\phi$).


# Pregunta 6: Redes Neuronales con Pytorch (3 puntos) 💬

<center>
<img src="https://www.anda.cl/wp-content/uploads/2021/03/0_5vNAtimPjYQr4W72.gif" alt="chatbot" width="400">
</center>

En esta sección de la tarea deberán implementar un Chatbot que sea capaz de generar una conversación *“básica”* utilizando un dataset de *Star Wars*. **El objetivo** de esta pregunta es que puedan aplicar lo aprendido sobre redes neuronales utilizando Pytorch en un ejemplo práctico.  Durante el desarrollo, se espera que puedan diseñar un bot (que tendrá por atrás un clasificador) que sea capaz de clasificar diferentes etiquetas, cosa que una vez identificada la etiqueta entregue una respuesta acorde a lo preguntado.

**Aviso:** Antes de comenzar con una descripción mas profunda de esta sección, les recomendamos que visualicen y se familiaricen con el dataset entregado, de esta forma comprenderán mejor la descripción del enunciado (aquí una pequeña ayudita 🆘).

In [ ]:
import pandas as pd

example_data = pd.read_json('https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/star_wars_chatbot.json')
print("Cantidad de tags: ", example_data['intents'].shape[0])

Cantidad de tags:  16


A continuación, ejemplos del contenido del primer registro:

In [ ]:
example_data['intents'][0]['patterns']

['Hi',
 'Hey',
 'How are you',
 'Is anyone there?',
 'Hello',
 'Good day',
 "What's up",
 'Yo!',
 'Howdy',
 'Nice to meet you.']

In [ ]:
example_data['intents'][0]['responses']

['Hey',
 'Hello, thanks for visiting.',
 'Hi there, what can I do for you?',
 'Hi there, how can I help?',
 'Hello, there.',
 'Hello Dear',
 'Ooooo Hello, looking for someone or something?',
 'Yes, I am here.',
 'Listening carefully.',
 'Ok, I am with you.']

In [ ]:
example_data['intents'][0]['tag']

'greeting'

In [ ]:
for i in range(len(example_data)):
  print(example_data['intents'][i]['tag'])

greeting
goodbye
thanks
tasks
alive
Menu
help
mission
jedi
sith
bounti hounter
funny
about me
creator
myself
stories


In [ ]:
example_data

,intents
0,"{'tag': 'greeting', 'patterns': ['Hi', 'Hey', ..."
1,"{'tag': 'goodbye', 'patterns': ['Bye', 'See yo..."
2,"{'tag': 'thanks', 'patterns': ['Thanks', 'Than..."
3,"{'tag': 'tasks', 'patterns': ['What can you do..."
4,"{'tag': 'alive', 'patterns': ['Are you alive.'..."
5,"{'tag': 'Menu', 'patterns': ['Which items do y..."
6,"{'tag': 'help', 'patterns': ['I am looking for..."
7,"{'tag': 'mission', 'patterns': ['I am on missi..."
8,"{'tag': 'jedi', 'patterns': ['Tell me top 10 j..."
9,"{'tag': 'sith', 'patterns': ['Tell me top 10 s..."


Del dataset cargado podemos notar que este viene en un formato `JSON`, por lo que sus datos están almacenados en diccionarios. Las llaves de los diccionarios no son aleatorias y estos nos sirven para identificar puntos relevantes en el desarrollo del bot. A continuación, se realiza una pequeña descripción de las llaves:

- `patterns`: Almacena los patrones con los que entrenaremos el modelo 😮, en otras palabras, es el corpus de entrenamiento que contiene solo preguntas o expresiones que deberá responder el bot.
- `responses`: Son las respuestas 🙋 relacionadas a los `patterns`, estas las utilizaremos en una etapa posterior a la clasificación, para dar una respuesta aleatoría al usuario.
- `tag`: Son las labels con las que entrenaremos nuestro modelo 💻.

En síntesis, las `keys` relevantes para el entrenamiento de nuestra red neuronal serán `patterns` (corpus) y `tag` (etiquetas).

#### Explicación de la tarea a realizar:

**Explicación de la tarea a realizar**: Implemente una Class llamada `CNNClassifier` que sea capaz de entrenar un modelo de texto a través de una red neuronal Feed Forward y una arquitectura convolucional (CNN 1D) [`torch.nn.Conv1d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html#conv1d) . Para el diseño de las redes tienen completa libertad, pero se le aconseja que se guíen de la última auxiliar para la construcción. Es **importantísimo** que el modelo a crear posea una capa de `Embedding` que se genere en base al entrenamiento del modelo. Creado el modelo, construya una función batch para cargar los datos de entrenamiento del modelo.

Construido el modelo, compare los resultados obtenidos para una red feed forward y una cnn. Para la comprobación de sus resultados ejecute el chatbot y pruebelo, ¿qué configuración tiene mejores resultados?, ¿a qué se deberan estos resultados?

Ojo que un ejemplo de prueba con el chatbot puede ser (agregue mas preguntas ud):

```
Let's chat! (type 'finish_chat' to finish the chat)
You: hi
GA-97: Yes, I am here.
You: can you tell me a joke?
GA-97: Have you tried the gluten-free Wookiee treats? No, but I heard they are a little Chewy.
```

El resto del código referido a la ejecución del chat se los entregamos, por lo que no deberían tener mayores problemas 😸 (en caso de tener problemas con su código, puede modificar cualquier parte sugerida siempre y cuando cumpla lo solicitado).

**Igual [mucho texto](https://i0.wp.com/elgeneracionalpost.com/wp-content/uploads/2020/07/mucho-texto.jpg?fit=1280%2C720&ssl=1).... En resumen, ¿Qué se solicita?:**

- [ ] Diseñar una red neuronal Feed Forward.
- [ ] Diseñar un red convolucional.
- [ ] Utilizar una capa de embeddings para generar representaciones vectoriales del corpus.
- [ ] Crear el método forward de la clase `CNNClassifier`.
- [ ] Crear la función BATCH.
- [ ] Probar el modelo y comparar los resultados obtenidos con la red Feed Forward y la red CNN. Comente sus resultados de forma cualitativa, señalando con qué tipo de red obtuvo mejores resultados con el chatbot.

**Nota-1:** El modelo creado debe tener la opción de entrenar a traves de una feed forward y una CNN. Esto no significa que entrenará una FF y una CN, el modelo deberá recibir un booleano que especifique que tipo de red utilizará.

**Nota-2:** El dataset se descargará automáticamente en la sección `Carga de Dataset 📚`, no os preocupéis.

#### Pasemos al Código 🦾

Esqueleto propuesto (se **RECOMIENDA** que cambien **SOLO** la red neuronal y la función Batch) 🦴:

##### Instalamos librerias necesarias e importamos 😀

In [ ]:
# Esto toma su tiempo en ejecutarse
%%capture
!pip install torch==1.8.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torchtext==0.9.0

In [ ]:
import os
import sys
import json
import torch
import random
from random import choice
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torch.optim import SGD, lr_scheduler
from torch.utils.data import DataLoader
from torch.autograd import Variable

from itertools import zip_longest

import plotly.express as px

import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer

##### Carga de Dataset 📚

In [ ]:
# we obtain the dataset
!wget 'https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/star_wars_chatbot.json'

--2023-06-15 20:43:14--  https://raw.githubusercontent.com/dccuchile/CC6205/master/assignments/star_wars_chatbot.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14469 (14K) [text/plain]
Saving to: ‘star_wars_chatbot.json’

star_wars_chatbot.j 100%[===================>]  14.13K  --.-KB/s    in 0.001s  

2023-06-15 20:43:14 (14.4 MB/s) - ‘star_wars_chatbot.json’ saved [14469/14469]



In [ ]:
# Load the dataset using json
with open('star_wars_chatbot.json', 'r') as f:
    dataset = json.load(f)

# Create a vocab with the dataset and get the number of classes that have
tokenizer = get_tokenizer("basic_english")
vocab = build_vocab_from_iterator(tokenizer(x) for list_words in dataset['intents'] for x in list_words['patterns'])
num_classes = len(dataset['intents'])

# Define a list with the labels
labels = sorted(set([tag for tag in [intents['tag'] for intents in dataset['intents']]]))
# Define a train_list where we can find the info in the format: [(tag_0, text_0)...,(tag_n-1, text_n-1)]
train_list = [(labels.index(intents['tag']), text) for intents in dataset['intents'] for text in intents['patterns']]

#vocab.set_default_index(0)
vocab.insert_token('<pad>', 1)
#stoi = vocab.get_stoi()

In [ ]:
labels

['Menu',
 'about me',
 'alive',
 'bounti hounter',
 'creator',
 'funny',
 'goodbye',
 'greeting',
 'help',
 'jedi',
 'mission',
 'myself',
 'sith',
 'stories',
 'tasks',
 'thanks']

##### Creación del modelo (2 puntos en total)

In [20]:
# Construya el modelo
class CNNClassifier(nn.Module):
    """Clasificador con red neuronal convolucional.
    """
    def __init__(self, vocab_size, embed_dim=32, num_classes=10,
                 use_cnn=False, cnn_pool_channels=24, cnn_kernel_size=3):
      """Inicializar clase CNNClassifier.

      Parameters
      ----------
      vocab_size : int
          Tamaño del vocabulario.

      embed_dim : int
          dimensión de la capa de embeddings (vocab_size * embed_dim).

      num_classes : int
          Número de clases del output de la capa lineal.

      use_cnn : bool
          True indica que se debe usar cnn. False que no debe ser usada.

      cnn_pool_channels : int
          Número de canales de salida de la convolución.

      cnn_kernel_size : int
          Tamaño del kernel de la convolución.

      """
      super().__init__()


      #if self.use_cnn==True:
        # Usar red feed forward
      #else:

      self.use_cnn = use_cnn

      # Creamos la capa de embedding
      self.embedding = nn.Embedding(vocab_size, embed_dim)

      # Creamos la capa de convolución
      self.conv = nn.Conv1d(
            in_channels=1,
            out_channels=cnn_pool_channels,
            kernel_size=cnn_kernel_size * embed_dim,
            stride=embed_dim,
            padding=cnn_kernel_size * embed_dim
        )

      # Calculamos el tamaño de entrada de la capa lineal
      fc_in_size = cnn_pool_channels

      # Creamos la capa lineal
      self.fc = nn.Linear(fc_in_size, num_classes)

      # Inicializamos los pesos de las capas
      self.init_weights()


    def init_weights(self):
      """Inicia los pesos de la red.

        Parameters
        ----------
        self :
            Red neuronal convolucional.
        """

      # Definimos el rango de los valores iniciales de los pesos
      initrange = 0.5

      # Inicializamos los pesos de la capa de embedding
      self.embedding.weight.data.uniform_(-initrange, initrange)

      # Inicializamos los pesos de la capa lineal
      self.fc.weight.data.uniform_(-initrange, initrange)

      # Inicializamos los sesgos de la capa lineal en cero
      self.fc.bias.data.zero_()


    def forward(self, text, offsets):
      """realiza el traspaso de información entre neuronas (capas).

        Parameters
        ----------
        text : str
            Oración a pasar a tensor.

        offset : array
            Índices para transformar a tensor el texto.

        Returns
        -------
        Tensor
            Resultado final mediante función de activación log softmax.
        """

      # Preparamos el input de la capa de embeddings a partir de text y offsets
      text = torch.tensor(
          list(
              zip(
                  *zip_longest(
                      *([text[o:offsets[i+1]] for i, o in enumerate(offsets[:-1])] + [text[offsets[-1]:len(texts)]]),
                      fillvalue=vocab["<pad>"]
                  )
              )
          )
      ).to(text.device)

      # Obtenemos la representación de la frase a partir de la capa de embedding
      h = self.embedding(text)

      # Aplicamos la capa de convolución
      h = h.view(h.size(0), 1, -1)
      h = torch.relu(self.conv(h))
      h = h.mean(dim=2)

      # Obtenemos el resultado final a partir de la capa lineal
      output = self.fc(h)

      # Aplicamos la función de activación log-softmax
      return F.log_softmax(output, dim=1)

##### Función Batch 👷 (0,5 puntos)

In [23]:
# Defina su función de BATCH
def generate_batch(batch):
  """Genera un subconjunto de datos.

  Parameters
  ----------
  batch : list of tuples
      Objeto a tokenizar, transformando a tensor junto con sus clases.

  Returns
  -------
  big_text: Tensor
      Texto tokenizado y transformado a tensor.

  offsets: Tensor
    Índices para transformar a tensor.

  label: Tensor
    Clase a la que pertenece la oración.
  """
  label = torch.tensor([entry[0] for entry in batch])
  texts = [tokenizer(entry[1]) for entry in batch]
  offsets = [0] + [len(text) for text in texts]
  offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
  #big_text = torch.cat([torch.tensor([vocab[t] if t in stoi else 0 for t in text]) for text in texts])
  big_text = torch.cat([torch.tensor([vocab.get_stoi()[t] for t in text]) for text in texts])

  return big_text, offsets, label

  #el perro labra y corre mucho-> pronombre, sujeto, verbo

##### Entrenamiento 🥊

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"GPU is avaible: {device}")

# Define the different inputs in our model
num_epochs = 1000
BATCH_SIZE = 16
LR = 1e-1
INPUT_SIZE = len(vocab)
OUTPUT_SIZE = num_classes
USE_CNN = False

# Define model, optimizer, loss and scheduler (Q: ¿What is it?)
model = CNNClassifier(INPUT_SIZE, num_classes=OUTPUT_SIZE, use_cnn=USE_CNN).to(device)
optimizer = SGD(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss().to(device)
scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda epoch: .9 ** (epoch // 10)])

print(f'train: {len(train_list)} elements')

# We train the model using the intents
loss_list= []
for epoch in range(1, num_epochs):
  train_loader = DataLoader(train_list, batch_size=BATCH_SIZE, collate_fn=generate_batch)
  model.train()
  total_loss = 0
  for i, (texts, offsets, cls) in enumerate(train_loader):
    texts = texts.to(device)
    offsets = offsets.to(device)
    cls = cls.to(device)
    optimizer.zero_grad()
    output = model(texts, offsets)
    loss = criterion(output, cls)
    total_loss += loss.item()
    loss.backward()
    optimizer.step()

  loss_list.append(loss.item())
  sys.stdout.write('\rEpoch: {0:03d} \t iter-Loss: {1:.3f}'.format(epoch+1, loss.item()))

print(f'final loss: {loss.item():.4f}')

GPU is avaible: cpu
train: 97 elements
Epoch: 1000 	 iter-Loss: 0.000final loss: 0.0001


##### A probar! 🧪

In [26]:
# This is working?, Try the next example!
qText = "'Do you know any joke?'" # this must classify the label "funny"

X = torch.tensor([vocab.get_stoi()[t] for t in tokenizer(qText)]).to(device)

model.eval()
output = model(X, torch.tensor([0], dtype=torch.long).to(device))
_, predicted = torch.max(output, dim=1)
labels[predicted]

'funny'

Ya pero prometiste hacer un chatbot, no una simple clasificación....

##### Guardamos modelo 🦺 (opcional)

In [27]:
# We save de model using pytorch (this is optional, just to learn how to do this in pytorch)
data = {
"model_state": model.state_dict(),
"input_size": INPUT_SIZE,
"output_size": OUTPUT_SIZE,
"use_cnn": USE_CNN,
"labels": labels
        }

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. file saved to {FILE}')

training complete. file saved to data.pth


##### Chatbot 💬

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('star_wars_chatbot.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

INPUT_SIZE = data["input_size"]
OUTPUT_SIZE = data["output_size"]
USE_CNN = data["use_cnn"]
labels = data['labels']
model_state = data["model_state"]

model = CNNClassifier(INPUT_SIZE, num_classes=OUTPUT_SIZE, use_cnn=USE_CNN).to(device)
model.load_state_dict(model_state)
model.eval()

# Dictionary with the answers
responses = {key['tag']: key['responses'] for key in dataset['intents']}

bot_name = "GA-97"
print("Let's chat! (type 'finish_chat' to finish the chat)")
while True:
    q_text = input("You: ")
    q_text = q_text
    if q_text == 'finish_chat':
        break

    X = torch.tensor([vocab.get_stoi()[t] for t in tokenizer(q_text)]).to(device)
    output = model(X, torch.tensor([0], dtype=torch.long).to(device))
    _, predicted = torch.max(output, dim=1)

    tag = labels[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.50:
      print(f"{bot_name}: {random.choice(responses[tag])}")
    else:
      print(f"{bot_name}: My model can't understand you...")

Let's chat! (type 'finish_chat' to finish the chat)
You: hey!
GA-97: Hello, there.
You: how are you?
GA-97: Hey
You: what's your name?


KeyError: ignored

#### Comente los resultados aquí (0,5 puntos)

Como se ha podido ver en esta tarea, se ha podido trabajar de buena manera el problema de tagging, junto con redes neuronales (convolucionales y recurrentes). Por otro lado, en la implementación de este chat-bot han aparecido carencias, ya sea en la definición de las redes o en la función de batch a usar. Sin embargo, se considera que va por buen camino dado que el bot ha sido capaz de responder cosas, sin tanto sentido pero indicando que la implementacoón va por buen camino.

Con respecto a los datos, se piensa que son pocos considerando que esta máquina debe aprender a responder un mayor conjunto de preguntas del usuario.